
# Customer Churn Prediction — Modeling, Selection & Explainability

**Objective:** Build, select, and explain a predictive model for customer churn using preprocessed datasets.  
**Deliverables:** Model selection table, final test metrics, ROC/PR curves, SHAP explainability, and a persisted final model.

**Setup:**  
- **Inputs** (already preprocessed): `data/processed/X_train.parquet`, `y_train.parquet`, `X_eval.parquet`, `y_eval.parquet`, `X_test.parquet`, `y_test.parquet`  
- **Outputs:**  
  - `reports/model_selection_results.csv`, `reports/test_metrics.json`  
  - `reports/figures/` (ROC, PR, confusion matrix, SHAP)  
  - `models/final_<model>.joblib`


## Environment & Imports

In [ ]:

import os, json, random
from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import (
    f1_score, precision_score, recall_score, accuracy_score, roc_auc_score,
    confusion_matrix, classification_report, roc_curve, precision_recall_curve
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb
import shap
import joblib
import matplotlib.pyplot as plt

# Reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# I/O
DATA_DIR = Path("data/processed")
REPORTS_DIR = Path("reports")
FIG_DIR = REPORTS_DIR / "figures"
MODELS_DIR = Path("models")
for d in [REPORTS_DIR, FIG_DIR, MODELS_DIR]:
    d.mkdir(parents=True, exist_ok=True)

print("Environment ready.")

## Load Data

In [ ]:

X_train = pd.read_parquet(DATA_DIR / "X_train.parquet")
y_train = pd.read_parquet(DATA_DIR / "y_train.parquet").squeeze()

X_eval  = pd.read_parquet(DATA_DIR / "X_eval.parquet")
y_eval  = pd.read_parquet(DATA_DIR / "y_eval.parquet").squeeze()

X_test  = pd.read_parquet(DATA_DIR / "X_test.parquet")
y_test  = pd.read_parquet(DATA_DIR / "y_test.parquet").squeeze()

print("Shapes:")
print("  X_train:", X_train.shape, " X_eval:", X_eval.shape, " X_test:", X_test.shape)
print("Positive rate (train/eval/test):",
      y_train.mean().round(3), y_eval.mean().round(3), y_test.mean().round(3))

## Sanity Checks

In [ ]:

# Basic feature sanity
assert set(X_train.columns) == set(X_eval.columns) == set(X_test.columns), "Column mismatch across splits"
assert len(X_train) == len(y_train) and len(X_eval) == len(y_eval) and len(X_test) == len(y_test), "Length mismatch"

# Class imbalance overview
class_imbalance = pd.DataFrame({
    "split": ["train", "eval", "test"],
    "positive_rate": [y_train.mean(), y_eval.mean(), y_test.mean()]
})
class_imbalance

## Cross-Validation Strategy & Scoring

In [ ]:

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
SCORING = "f1"
N_JOBS  = -1
N_ITER  = 30   # runtime-controlled exploration
VERBOSE = 2

## Hyperparameter Search Spaces

In [ ]:

# Logistic Regression (preprocessed data, no pipeline needed here)
param_dist_lr = {
    "C": [0.001, 0.01, 0.1, 1, 10],
    "penalty": ["l2"],
    "solver": ["liblinear", "lbfgs"],
    "class_weight": [None, "balanced"]
}

# Random Forest
param_dist_rf = {
    "n_estimators": [100, 300, 500],
    "max_depth": [None, 10, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2"],
    "class_weight": [None, "balanced", "balanced_subsample"]
}

# XGBoost
param_dist_xgb = {
    "n_estimators": [100, 300, 500],
    "max_depth": [3, 6, 9],
    "learning_rate": [0.01, 0.1],
    "subsample": [0.8, 1.0],
    "colsample_bytree": [0.8, 1.0],
    "gamma": [0, 1, 5],
    "min_child_weight": [1, 5],
    "reg_alpha": [0, 0.1],
    "reg_lambda": [1, 5, 10]
}

## Model Training (RandomizedSearchCV on Train)

In [ ]:

searches = {}

# Logistic Regression
lr = LogisticRegression(max_iter=500, random_state=SEED)
search_lr = RandomizedSearchCV(
    lr, param_distributions=param_dist_lr, n_iter=N_ITER,
    scoring=SCORING, cv=cv, n_jobs=N_JOBS, verbose=VERBOSE, random_state=SEED
)
search_lr.fit(X_train, y_train)
searches["LogisticRegression"] = search_lr

# Random Forest
rf = RandomForestClassifier(random_state=SEED)
search_rf = RandomizedSearchCV(
    rf, param_distributions=param_dist_rf, n_iter=N_ITER,
    scoring=SCORING, cv=cv, n_jobs=N_JOBS, verbose=VERBOSE, random_state=SEED
)
search_rf.fit(X_train, y_train)
searches["RandomForest"] = search_rf

# XGBoost
xgb_clf = xgb.XGBClassifier(random_state=SEED, eval_metric="logloss", verbosity=1)
search_xgb = RandomizedSearchCV(
    xgb_clf, param_distributions=param_dist_xgb, n_iter=N_ITER,
    scoring=SCORING, cv=cv, n_jobs=N_JOBS, verbose=VERBOSE, random_state=SEED
)
search_xgb.fit(X_train, y_train)
searches["XGBoost"] = search_xgb

print("Searches completed.")

## Model Comparison on Eval Set

In [ ]:

def evaluate_on_eval(name, search, X_eval, y_eval):
    best_est = search.best_estimator_
    y_pred = best_est.predict(X_eval)
    if hasattr(best_est, "predict_proba"):
        y_proba = best_est.predict_proba(X_eval)[:, 1]
    else:
        # fallback: probability-free models
        y_proba = None

    out = {
        "model": name,
        "cv_f1": round(search.best_score_, 4),
        "eval_f1": round(f1_score(y_eval, y_pred), 4),
        "eval_precision": round(precision_score(y_eval, y_pred), 4),
        "eval_recall": round(recall_score(y_eval, y_pred), 4),
        "eval_accuracy": round(accuracy_score(y_eval, y_pred), 4),
        "best_params": search.best_params_
    }
    if y_proba is not None:
        out["eval_roc_auc"] = round(roc_auc_score(y_eval, y_proba), 4)
    return out

results = [evaluate_on_eval(n, s, X_eval, y_eval) for n, s in searches.items()]
results_df = pd.DataFrame(results).sort_values("eval_f1", ascending=False).reset_index(drop=True)
results_df

## Retrain Best Model on Train+Eval and Evaluate on Test

In [ ]:

# Winner by eval_f1
winner_name = results_df.iloc[0]["model"]
winner_params = searches[winner_name].best_params_
print(f"Winner: {winner_name}\nParams: {winner_params}")

# Merge train + eval for final training
X_full = pd.concat([X_train, X_eval], axis=0).reset_index(drop=True)
y_full = pd.concat([y_train, y_eval], axis=0).reset_index(drop=True)

# Instantiate final model
if winner_name == "LogisticRegression":
    final_model = LogisticRegression(max_iter=500, random_state=SEED, **winner_params)
elif winner_name == "RandomForest":
    final_model = RandomForestClassifier(random_state=SEED, **winner_params)
else:
    final_model = xgb.XGBClassifier(random_state=SEED, eval_metric="logloss", **winner_params)

# Fit & predict
final_model.fit(X_full, y_full)
y_pred_test = final_model.predict(X_test)
y_proba_test = final_model.predict_proba(X_test)[:, 1] if hasattr(final_model, "predict_proba") else None

# Metrics
test_metrics = {
    "test_f1": f1_score(y_test, y_pred_test),
    "test_precision": precision_score(y_test, y_pred_test),
    "test_recall": recall_score(y_test, y_pred_test),
    "test_accuracy": accuracy_score(y_test, y_pred_test)
}
if y_proba_test is not None:
    test_metrics["test_roc_auc"] = roc_auc_score(y_test, y_proba_test)

print(pd.Series(test_metrics).round(4))

# Confusion Matrix (print + plot)
cm = confusion_matrix(y_test, y_pred_test)
print("Confusion matrix:\n", cm)

print("\nClassification report:\n", classification_report(y_test, y_pred_test))

## Plots — ROC, Precision-Recall, and Confusion Matrix

In [ ]:

# ROC Curve
if y_proba_test is not None:
    fpr, tpr, _ = roc_curve(y_test, y_proba_test)
    plt.figure(figsize=(6,4))
    plt.plot(fpr, tpr, label="ROC")
    plt.plot([0,1], [0,1], linestyle="--")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve (Test)")
    plt.legend()
    roc_path = FIG_DIR / "roc_curve_test.png"
    plt.tight_layout(); plt.savefig(roc_path, dpi=150); plt.close()
    print("Saved:", roc_path)

# PR Curve
if y_proba_test is not None:
    precision, recall, _ = precision_recall_curve(y_test, y_proba_test)
    plt.figure(figsize=(6,4))
    plt.plot(recall, precision, label="PR")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title("Precision-Recall Curve (Test)")
    plt.legend()
    pr_path = FIG_DIR / "pr_curve_test.png"
    plt.tight_layout(); plt.savefig(pr_path, dpi=150); plt.close()
    print("Saved:", pr_path)

# Confusion Matrix Heatmap (simple)
plt.figure(figsize=(4,4))
# No custom colors per your plotting constraints
plt.imshow(cm, interpolation="nearest")
plt.title("Confusion Matrix (Test)")
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks(tick_marks, ["No", "Yes"], rotation=45)
plt.yticks(tick_marks, ["No", "Yes"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, cm[i, j], ha="center", va="center")
cm_path = FIG_DIR / "confusion_matrix_test.png"
plt.tight_layout(); plt.savefig(cm_path, dpi=150); plt.close()
print("Saved:", cm_path)

## Persist Artifacts

In [ ]:

# Model selection table & test metrics
results_df.to_csv(REPORTS_DIR / "model_selection_results.csv", index=False)
with open(REPORTS_DIR / "test_metrics.json", "w") as f:
    json.dump({k: float(v) for k, v in test_metrics.items()}, f, indent=2)
print("Saved model selection and test metrics.")

# Final model
model_path = MODELS_DIR / f'final_{winner_name.lower().replace(" ", "_")}.joblib'
joblib.dump(final_model, model_path)
print("Saved model to:", model_path)

## Explainability with SHAP

In [ ]:

# Tree-based -> TreeExplainer; otherwise KernelExplainer (slower)
is_tree = isinstance(final_model, (RandomForestClassifier, xgb.XGBClassifier))

if is_tree:
    explainer = shap.TreeExplainer(final_model)
    shap_values = explainer.shap_values(X_test)

    # Summary (positive class for binary)
    plt.figure(figsize=(10,5))
    if isinstance(shap_values, list) and len(shap_values) == 2:
        shap.summary_plot(shap_values[1], X_test, show=False)
    else:
        shap.summary_plot(shap_values, X_test, show=False)
    shap_sum_path = FIG_DIR / "shap_summary.png"
    plt.tight_layout(); plt.savefig(shap_sum_path, dpi=150); plt.close()
    print("Saved:", shap_sum_path)

    # Importance bar (mean |SHAP|)
    plt.figure(figsize=(10,5))
    if isinstance(shap_values, list) and len(shap_values) == 2:
        shap.summary_plot(shap_values[1], X_test, plot_type="bar", show=False)
    else:
        shap.summary_plot(shap_values, X_test, plot_type="bar", show=False)
    shap_bar_path = FIG_DIR / "shap_importance_bar.png"
    plt.tight_layout(); plt.savefig(shap_bar_path, dpi=150); plt.close()
    print("Saved:", shap_bar_path)

else:
    # KernelExplainer with small background for speed
    background = X_train.sample(min(500, len(X_train)), random_state=SEED)
    explainer = shap.KernelExplainer(final_model.predict_proba, background)
    sample_X = X_test.sample(min(1000, len(X_test)), random_state=SEED)
    shap_values = explainer.shap_values(sample_X)

    plt.figure(figsize=(10,5))
    shap.summary_plot(shap_values[1], sample_X, show=False)
    shap_sum_path = FIG_DIR / "shap_summary.png"
    plt.tight_layout(); plt.savefig(shap_sum_path, dpi=150); plt.close()
    print("Saved:", shap_sum_path)


## Next Steps
- Threshold tuning to hit business targets (e.g., maximize recall at fixed precision).
- Segment-level error analysis (e.g., by Contract, Tenure).
- Calibrated probabilities (`CalibratedClassifierCV`) if required by downstream systems.
- Packaging for deployment (batch/API) and monitoring (data drift, performance drift).
